# Исследование надежности заемщиков


**Цель:** Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

## Откройте таблицу и изучите общую информацию о данных

**1. Импортируйте библиотеку pandas. Считайте данные из csv-файла в датафрейм и сохраните в переменную `data`.

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/datasets/data.csv')

**2. Выведите первые 20 строчек датафрейма `data` на экран.**

In [ ]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**3. Выведите основную информацию о датафрейме с помощью метода `info()`.**

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**4. Выведите количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов.**

In [ ]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**5. Заполните пропуски в столбце `total_income` медианным значением по каждому типу из столбца `income_type`.**

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**6. В столбце `days_employed` замените все отрицательные значения положительными с помощью метода `abs()`.**

In [ ]:
data['days_employed'] = data['days_employed'].abs()

**7. Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.**

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится нам для исследования.

**8. Выведите перечень уникальных значений столбца `children`.**

In [ ]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**9. В столбце `children` есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**10. Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [ ]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**11. Заполните пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**12. Убедитесь, что все пропуски заполнены. Проверьте себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.**

In [ ]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**13. Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [ ]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**14. Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру. Проверьте остальные столбцы.**

In [ ]:
data['education'] = data['education'].str.lower()

**15. Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**

In [ ]:
data.duplicated().sum()

71

In [ ]:
data = data.drop_duplicates()

### Категоризация данных

**16. На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**17. Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**18. Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.


In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
print(data.isna().sum())

children                 0
days_employed            0
dob_years                0
education                0
education_id             0
family_status            0
family_status_id         0
gender                   0
income_type              0
debt                     0
total_income             0
purpose                  0
total_income_category    0
purpose_category         0
dtype: int64


### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [ ]:
data.loc[data['children'] >= 3, 'children'] = 'many_kids'
data_children = data.pivot_table(index=['children'], values='debt', aggfunc=['count', 'sum', 'mean'])
data_children.columns=['total', 'in_debt', 'percentage']
data_children['percentage'] = data_children['percentage'] * 100
print(data_children)

           total  in_debt  percentage
children                             
0          14091     1063    7.543822
1           4808      444    9.234609
2           2052      194    9.454191
many_kids    380       31    8.157895


**Вывод:** Сгруппировав данные о клиентах по количеству детей, мы поняли, что количество бездетных с кредитами в 2 раза больше чем количество всех семей с детьми, кто взял кредит. В то же время, процент задолженности растёт с количеством детей (не берём в рассчёт многодетные семьи, т.к. их количество очень маленькое и не может считаться на 100% достоверным). Стоит отметить, что есть относительно существенная разница по задолженностям между семьями с детьми и без детей, однако, между семьями с детьми разница по задолженности уже не такая большая. Можно сделать вывод о том, что количество детей влияет на вероятность возврата кредита в срок, но более решающим фактором можно считать именно факт наличия детей, т.к. именно он показывает большую разницу в вероятности долга.


#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
data_family = data.pivot_table(index=['family_status'], values='debt', aggfunc=['count', 'sum', 'mean'])
data_family.columns = ['total', 'in_debt', 'percentage']
data_family['percentage'] = data_family['percentage'] * 100
data_family = data_family.sort_values(by='total', ascending=False)
print(data_family)

                       total  in_debt  percentage
family_status                                    
женат / замужем        12261      927    7.560558
гражданский брак        4134      385    9.313014
Не женат / не замужем   2796      273    9.763948
в разводе               1189       84    7.064760
вдовец / вдова           951       63    6.624606


**Вывод:** Данные по семейному положению показывают, что наиболее вероятна задолженность у людей, которые не имеют или не имели официального супруга/супруги. Процент долга у людей в разводе и вдов/вдовцов меньше, но выборка небольшая и может быть не на 100% достоверной. Наиболее стабильное положение показывают люди в браке, процент долга ниже (относительно имеющихся данных) и их выборка самая большая.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
data_income = data.pivot_table(index=['total_income_category'], values='debt', aggfunc=['count', 'sum', 'mean'])
data_income.columns = ['total', 'in_debt', 'percentage']
data_income['percentage'] = data_income['percentage'] * 100
data_income = data_income.sort_values(by='total', ascending=False)
print(data_income)

                       total  in_debt  percentage
total_income_category                            
C                      15921     1353    8.498210
B                       5014      354    7.060231
D                        349       21    6.017192
A                         25        2    8.000000
E                         22        2    9.090909


**Вывод:** Данная выборка менее информативна, т.к. очень мало данных по категориям A, D и E - людям, которые получают самый высокий и самый низкий доход. Поэтому, в рассчёт берём только категории B и C. Мы видим, что процент долга у людей с доходом категории C выше, чем у людей с категорией B. В добавок, разница сумм дохода этих категорий довольна большая, поэтому можно сказать, что уровень дохода влияет на вероятность возврата кредита в срок. Однако, стоит расширить выборку по другим категориям, чтобы сделать окончательный вывод.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [ ]:
data_purpose = data.pivot_table(index=['purpose_category'], values='debt', aggfunc=['count', 'sum', 'mean'])
data_purpose.columns = ['total', 'in_debt', 'percentage']
data_purpose['percentage'] = data_purpose['percentage'] * 100
data_purpose = data_purpose.sort_values(by='total', ascending=False)
print(data_purpose)

                          total  in_debt  percentage
purpose_category                                    
операции с недвижимостью  10751      780    7.255139
операции с автомобилем     4279      400    9.347978
получение образования      3988      369    9.252758
проведение свадьбы         2313      183    7.911803


**Вывод:** По результатам анализа данных по целям получения кредита, можно сказать, что меньшая вероятность задолженности у людей, берущих кредит для операций с недвижимостью или проведением свадьбы. Большая вероятность у людей, получающих образование или приобретающих автомобили. Можно предположить, что семейный статус клиентов в данном случае также имеет влияние на вероятность долга, т.к. люди нередко приобретают недвижимость будучи в браке, что повышает платёжеспособность (тоже самое и в случае свадьбы).


#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

**Ответ:**
В таблице равное количество пропусков в столбцах 'total_income' и 'days_epmployed', по 2174 пропуска в каждом. После фильтрации датасета мы оставили только строки с пропусками, которых также оказалось 2174. Это говорит о том, что пропуски есть одновременно в двух столбцах. Вероятно, эти данные не были указаны намеренно, можно обратиться в коллегам, отвечающим за выгрузку данных, чтобы проверить этот факт.

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

**Ответ:** Зачастую, количественные данные несут важную для исследования информацию. Поэтому, их лучше заменять медианным значением, т.к. среднее значение(в случае неравномерной выборки) или 0 в качестве замены могут исказить рассчёты. Как и в данном случае, значения в столбцах days_employed и total_income критично важны, т.к. они влияют на конечные результаты рассчётов.

### Шаг 4: Общий вывод

Было проведено исследование о влиянии различных факторов на вероятность возврата клиентом кредита в срок. Факторы влияния были следущими:

**1. Влияет ли количество детей на возврат кредита в срок?**
Ответ: **Частично.** по результатам анализа данных по количеству детей, мы поняли, что данных по многодетным семьям (3 ребенка и более) недостаточно для достоверных выводов. По другим категориям мы выявили следующую зависимость:

7,54% - процент задолженностей по кредиту у клиентов без детей;
9,23% - процент задолженностей по кредиту у клиентов с 1 ребёнком;
9,45% - процент задолженностей по кредиту у клиентов с 2 детьми.

Факт наличия детей действительно влияет на вероятность долга, однако, стоит проанализировать данные по многодетным семьям для более достоверной картины.

**2. Влияет ли семейное положение на возврат кредита в срок?**
Ответ: **Да**, после анализа мы получили следующие показатели:

7,5% и ниже - процент вероятности долга у категорий 'замужем/женат','вдова/вдовец','в разводе';
9,3% и выше - процент вероятности долга у категорий 'не женат/незамужем','гражданский брак'.

Результаты показывают, что люди находящиеся или когда-либо состоящие в официальном браке являются более стабильной категорией клиентов.

**3. Влияет ли уровень дохода на возврат кредита в срок?**
Ответ: **Частично.** Для полной картины не хватает данных о категориях D(от 30 001 до 50 000), E(до 30 000) и A (от 1 000 001). По другим категориям мы получили следующий процент:

7,06% - процент вероятности долга у клиентов с доходом от 200 001 до 1 000 000;
8,49% - процент вероятности долга у клиентов с доходом от 50 001 до 200 000.

В данных случаях уровень дохода имеет влияние, но вывод неполноценный ввиду отсутсвия достаточного количества данных по другим категориям.

**4. Как разные цели влияют на погашение кредита в срок?**
Ответ: **Да**, цели кредита влияют на вероятность его погашения в срок.

Категории с наименьшей вероятностью просрочки по кредиту:
7,25% - операции с недвижимостью;
7,91% - проведение свадьбы.

Категории с наибольшей вероятностью просрочки по кредиту:
9,25% - получение образования;
9,34% - операции с автомобилем.

**Общий вывод:**
Для построение системы скоринга стоит учитывать фактор семейного положения и целей кредита.
    Стоит взять больше данных для окончательных выводов по факторам уровня дохода и количества детей.